In [30]:
import os
import subprocess
import Bio
import sys
import argparse
import pathlib
from pathlib import Path
import tempfile
from Bio import SeqIO, SearchIO
import pandas as pd

In [6]:
#Define what a directory path is

#Parse arguments       
parser = argparse.ArgumentParser(description='Database named after .protein.fa')
parser.add_argument('-i', '--query', type=str, help='Query sequences')
parser.add_argument('-c', '--db', type=pathlib.Path, help='Directory of database')
parser.add_argument('-e', '--eval', type=float, help='Blast e-value')
parser.add_argument('-q', '--qcov', type=str, help='Query coverage')
parser.add_argument('-bo', '--blasto', type=str, help='Blast output')
parser.add_argument('-o', '--fasta', type=str, help='Name and sequence from Blast')

args = parser.parse_args()


usage: ipykernel_launcher.py [-h] [-i QUERY] [-c DB] [-e EVAL] [-q QCOV]
                             [-bo BLASTO]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/honghaosu/.local/share/jupyter/runtime/kernel-5f2b2d96-a76c-4f6d-a5a0-c74657f6d472.json


SystemExit: 2

/home/honghaosu/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3259: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
#Concatenate all peps into the same variable
print("Just for the first round of mining")
fasta_dir = args.db
fasta_files = [f for f in os.listdir(fasta_dir) if f.endswith(".protein.fa")]

database = ''

for fasta_file in fasta_files:
    with open(os.path.join(fasta_dir, fasta_file), 'r') as f:
        database += f.read()


In [ ]:
#makeblastdb
with tempfile.NamedTemporaryFile(mode="w", delete=False) as tmp:
    tmp.write(database)
    tmp_path = tmp.name

def make_blastdb(db_input, db_output_dir):
    db_output_dir_str = str(db_output_dir)
    cmd = ["makeblastdb", "-in", db_input, "-dbtype", "prot", "-out", f"{db_output_dir_str}/blastpdb"]
    subprocess.run(cmd)
    print("makeblastdb completed")

make_blastdb(tmp_path, args.db)

os.remove(tmp_path)


In [ ]:
#Blastp
query_file = args.query
db_file = f"{args.db}/blastpdb"
output_file = args.blasto
e_value = args.eval
qcov_hsp_perc = args.qcov

cmd = ["blastp", "-query", query_file, "-db", db_file, "-out", output_file, "-evalue", str(e_value), "-qcov_hsp_perc", str(qcov_hsp_perc), "-outfmt", "7"]
subprocess.run(cmd, check=True)

In [52]:
#Filter

  
process_cmd=['sed', '-e', '/^#/d', output_file] 
filtered=subprocess.check_output(process_cmd)

# Write the output to a new file
tmp="temp.tsv" 
with open(tmp, 'wb') as f:
    f.write(filtered)

blast_df=pd.read_table("temp.tsv", header=None)
print(blast_df.head(10))
blast_df_filtered=blast_df[blast_df[3]>600]#May make this an argument
print("Sequences with amino acid length lower than 600 are removed")
#print(blast_df_filtered.head(10))

blast_sequence=blast_df_filtered[1]
#print(blast_sequence.head(10))
blast_sequence_unique=list(set(blast_sequence))
#print(blast_sequence_unique)

os.remove(tmp)

      0                    1       2    3    4   5   6    7   8    9    10  \
0  AabAS   Solyc12g006530.2.1  78.844  761  161   0   1  761   1  761  0.0   
1  AabAS  Glyma.07G001300.1.p  77.483  755  169   1   1  754   1  755  0.0   
2  AabAS  Glyma.07G001300.2.p  77.556  753  168   1   1  752   1  753  0.0   
3  AabAS  Glyma.08G225800.1.p  77.748  755  167   1   1  754   1  755  0.0   
4  AabAS   Bevul.7G149700.1.p  76.587  756  174   3   1  754   1  755  0.0   
5  AabAS   Solyc12g006505.1.1  74.113  761  197   0   1  761  21  781  0.0   
6  AabAS  Glyma.15G065600.2.p  70.503  756  222   1   1  755   1  756  0.0   
7  AabAS          AT1G78955.1  71.693  756  212   2   1  754   1  756  0.0   
8  AabAS          AT1G78950.1  72.222  756  208   2   1  754   1  756  0.0   
9  AabAS     Brara.G03624.1.p  70.503  756  221   2   1  754   1  756  0.0   

     11  
0  1297  
1  1270  
2  1268  
3  1266  
4  1249  
5  1232  
6  1172  
7  1172  
8  1171  
9  1155  
      0                    1   

In [50]:
#Make a dictionary
with tempfile.NamedTemporaryFile(mode="w", delete=False) as tmp:
    tmp.write(database)
    tmp_path = tmp.name

    
def parse_fasta(fasta_file):
    """
    Parses a FASTA file and returns a dictionary of sequences.
    """
    sequences_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    return sequences_dict

sequences_dict = parse_fasta(tmp_path)

sequences = {}
for seq_id, seq_record in sequences_dict.items():
    sequences[seq_id] = str(seq_record.seq)
#The name of the dictionary is called sequences



Empty DataFrame
Columns: [Name, Sequence]
Index: []


In [69]:
#Search against dictionary
Pep_sequence=[]
for key in blast_sequence_unique:
    Pep_sequence.append(sequences[key])

#Combine sequence name and sequence

data_dict = {'Name': blast_sequence_unique, 'Sequence': Pep_sequence}

# Create the DataFrame
Final_result = pd.DataFrame(data_dict)
Final_result.to_csv('temp.tsv', sep='\t', index=False, header=False)
tsv=SeqIO.parse("temp.tsv", "tab")
count = SeqIO.write(tsv, args.fasta, "fasta")
print("Converted %i records" % count)

os.remove("temp.tsv")

In [58]:
query=args.query
new=args.fasta
cat_cmd=['cat', query, new]
cat_process=subprocess.run(cat_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
with open("merged.fasta", "w") as f:
    f.write(cat_process.stdout.decode())

mafft_cmd=['mafft', '--maxiterate', '1000','--globalpair', 'merged.fasta']
mafft_process = subprocess.run(mafft_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
mafft_output=args.fasta+".mafft"
if mafft_process.returncode != 0:
    print(f"MAFFT exited with an error: {mafft_process.stderr.decode()}")
else:
    # write the output to a file
    with open(mafft_output, "w") as f:
        f.write(mafft_process.stdout.decode())
    print(f"MAFFT finished successfully. Output written to {mafft_output}")
    
os.remove("merged.fasta")

MAFFT finished successfully. Output written to All-OSC-blastp.mafft


In [59]:
fasttree_cmd=['FastTree', mafft_output]
fasttree_process=subprocess.run(fasttree_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
fasttree_output=mafft_output+".fst"
if fasttree_process.returncode != 0:
    print(f"FastTree exited with an error: {fasttree_process.stderr.decode()}")
else:
    with open(fasttree_output, "w") as f:
        f.write(fasttree_process.stdout.decode())
    print(f"FastTree finished successfully. Output written to {fasttree_output}")

FastTree finished successfully. Output written to All-OSC-blastp.mafft.fst
